In [23]:
import numpy as np
import pandas as pd

import os

import pickle

import cv2
import dlib

from scipy.io import loadmat
from math import cos, sin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [24]:
path_of_the_directory = 'D:\\ITI\\Statistical ML 1\\project\\AFLW2000\\'
ext_mat = '.mat'
ext_img = '.jpg'
df = pd.DataFrame(columns = ['img', 
                            'Chin_x', 
                            'Chin_y', 
                            'Tip of the Nose_x', 
                            'Tip of the Nose_y', 
                            'Left side of the left eye_x',
                            'Left side of the left eye_y',
                            'Right side of the right eye_x',
                            'Right side of the right eye_y',
                            'Left side of the mouth_x',
                            'Left side of the mouth_y',
                            'Right side of the mouth_x',
                            'Right side of the mouth_y',
                            'yaw',
                            'pitch',
                            'roll'])

In [25]:
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext_img):
        name = file
    if file.endswith(ext_mat):
        annots = loadmat(os.path.join(path_of_the_directory + file))
        pt2d = annots['pt3d_68'][:2,:]

        pre_pose_params = annots['Pose_Para'][0]
        pose_params = pre_pose_params[:3] * 180 / np.pi

        params = [[name, 
                    pt2d[0,8], 
                    pt2d[1,8], 
                    pt2d[0,30], 
                    pt2d[1,30], 
                    pt2d[0,36], 
                    pt2d[1,36], 
                    pt2d[0,45], 
                    pt2d[1,45], 
                    pt2d[0,48],
                    pt2d[1,48], 
                    pt2d[0, 54], 
                    pt2d[1, 54],
                    pose_params[1],
                    pose_params[0],
                    pose_params[2]
                    ]]

        df_temp = pd.DataFrame(params,
                                columns = ['img', 
                                'Chin_x', 
                                'Chin_y', 
                                'Tip of the Nose_x', 
                                'Tip of the Nose_y', 
                                'Left side of the left eye_x',
                                'Left side of the left eye_y',
                                'Right side of the right eye_x',
                                'Right side of the right eye_y',
                                'Left side of the mouth_x',
                                'Left side of the mouth_y',
                                'Right side of the mouth_x',
                                'Right side of the mouth_y',
                                'yaw',
                                'pitch',
                                'roll'])

        df = pd.concat([df, df_temp], ignore_index=True)

In [26]:
df

,img,Chin_x,Chin_y,Tip of the Nose_x,Tip of the Nose_y,Left side of the left eye_x,Left side of the left eye_y,Right side of the right eye_x,Right side of the right eye_y,Left side of the mouth_x,Left side of the mouth_y,Right side of the mouth_x,Right side of the mouth_y,yaw,pitch,roll
0,image00002.jpg,213.940063,375.730499,225.088425,281.803406,162.579117,199.647583,301.851685,212.36467,180.507431,305.174622,261.812347,322.659698,1.044306,-22.874239,4.908885
1,image00004.jpg,207.085449,345.316071,186.151962,262.358002,219.600342,239.387238,256.144623,224.261322,206.686462,311.824158,226.800659,303.561066,68.155235,26.932741,17.24367
2,image00006.jpg,238.303162,351.412842,196.313782,273.822266,198.901047,239.487122,254.840668,231.438049,212.726471,304.789856,252.738708,300.910248,50.485409,-10.579652,-13.570644
3,image00008.jpg,268.327637,362.018036,216.827087,282.946289,166.38385,244.075317,268.555054,208.186081,213.172653,316.072205,269.946014,299.294373,17.143373,-10.048455,-21.39278
4,image00010.jpg,250.219299,356.444702,173.97554,282.408569,200.708405,243.682465,216.878983,227.281342,223.81958,312.089142,229.980194,310.60675,68.640549,-50.544579,-59.207973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,image04358.jpg,213.987427,372.599121,231.1082,279.537415,163.308197,206.959961,297.862183,218.591782,186.03511,315.108887,251.346939,321.491089,-4.035367,-11.293093,6.022806
1996,image04363.jpg,223.977646,374.818481,271.463501,276.808716,207.591187,232.862717,224.472351,226.686249,225.867645,322.267975,231.436157,317.13324,-81.288437,-13.327947,10.081746
1997,image04364.jpg,239.93924,377.36319,179.231552,274.731262,216.805939,231.459717,218.558716,226.749115,228.697906,312.43634,221.335007,317.766571,82.005554,-82.961678,-86.483292
1998,image04365.jpg,208.983047,374.496033,282.930267,265.670654,209.662598,209.398865,248.470062,215.537491,207.04068,310.097931,234.653503,313.753479,-68.249329,-24.090855,25.869925


In [27]:
df.to_csv('Complete Dataset.csv', index = False)

In [28]:
# READ THE NEW DATASET

df = pd.read_csv('Complete Dataset.csv')

In [29]:
# DRAW THE LINES

def draw_axis(img, yaw, pitch, roll, tdx = None, tdy = None, size = 100):

    pitch   = pitch * np.pi / 180
    yaw     = -(yaw * np.pi / 180)
    roll    = roll * np.pi / 180

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis pointing downwards drawn in green
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [30]:
# load
with open('SVM_YAW.pkl','rb') as f:
    regr_yaw = pickle.load(f)

with open('SVM_PITCH.pkl','rb') as f:
    regr_pitch = pickle.load(f)

with open('SVM_ROLL.pkl','rb') as f:
    regr_roll = pickle.load(f)

c:\Users\moham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\moham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVR from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\moham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.1.1.

In [39]:
cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)

result = cv2.VideoWriter('Pose Recognition.mp4', 
                         cv2.VideoWriter_fourcc(*'MPEG'),
                         10, size)

while cap.isOpened():
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        landmarks = predictor(gray, face)

        parts = landmarks.parts()

        # 6 Features extracted
        for part in [8, 30, 36, 45, 48, 54]:
            cv2.circle(frame, (landmarks.part(part).x, landmarks.part(part).y), 2, (255,0,0), 1)
        
        coordinates = np.array([[
                        parts[8].x,               #Chin
                        parts[8].y,
                        parts[30].x,              #Tip of the nose
                        parts[30].y,
                        parts[36].x,              #Left side of the left eye
                        parts[36].y,
                        parts[45].x,              #Right side of the right eye
                        parts[45].y,
                        parts[48].x,              #Left side of the mouth
                        parts[48].y,
                        parts[54].x,              #Right side of the mouth
                        parts[54].y,
                        ]])

        df = pd.DataFrame(coordinates, 
                        columns = [ 'Chin_x', 
                                    'Chin_y', 
                                    'Tip of the Nose_x', 
                                    'Tip of the Nose_y', 
                                    'Left side of the left eye_x',
                                    'Left side of the left eye_y',
                                    'Right side of the right eye_x',
                                    'Right side of the right eye_y',
                                    'Left side of the mouth_x',
                                    'Left side of the mouth_y',
                                    'Right side of the mouth_x',
                                    'Right side of the mouth_y',]
                                    )
        
        yaw_predicted = regr_yaw.predict(df)
        pitch_predicted = regr_pitch.predict(df)
        roll_predicted = regr_roll.predict(df) - 10

        # result.write(frame)
        
        cv2.imshow("Pose Estimation", draw_axis(frame, yaw_predicted[0],
                                                pitch_predicted[0],
                                                roll_predicted[0],
                                                df['Tip of the Nose_x'].tolist()[0],
                                                df['Tip of the Nose_y'].tolist()[0]))
        
        result.write(frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

KeyboardInterrupt: 

In [40]:
cap.release()
result.release()
print("The video was successfully saved")
cv2.destroyAllWindows()

The video was successfully saved
